In [1]:
from google.colab import files
uploaded = files.upload()

Saving Dataset - Estadísticas Jugadores.csv to Dataset - Estadísticas Jugadores.csv



# 1. **Bibliotecas y Dataset**

In [2]:
# Bibliotecas Necesarias:

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import random

# Elección, Carga y Estudio del Dataset:

dataset = pd.read_csv('Dataset - Estadísticas Jugadores.csv', encoding='latin1', sep=';')
dataset.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Off,Crs,TklW,PKwon,PKcon,OG,Recov,AerWon,AerLost,AerWon%
0,1,Max Aarons,ENG,DF,Norwich City,Premier League,22.0,2000,34,32,...,0.03,1.41,1.16,0.0,0.06,0.03,5.53,0.47,1.59,22.7
1,2,Yunis Abdelhamid,MAR,DF,Reims,Ligue 1,34.0,1987,34,34,...,0.00,0.06,1.39,0.0,0.03,0.00,6.77,2.02,1.36,59.8
2,3,Salis Abdul Samed,GHA,MF,Clermont Foot,Ligue 1,22.0,2000,31,29,...,0.00,0.36,1.24,0.0,0.00,0.00,8.76,0.88,0.88,50.0
3,4,Laurent Abergel,FRA,MF,Lorient,Ligue 1,29.0,1993,34,34,...,0.03,0.79,2.23,0.0,0.00,0.00,8.87,0.43,0.43,50.0
4,5,Charles Abi,FRA,FW,Saint-Étienne,Ligue 1,22.0,2000,1,1,...,0.00,2.00,0.00,0.0,0.00,0.00,4.00,2.00,0.00,100.0


# 2. **Preprocesamiento de Datos**

In [3]:
# Visualización de las Posiciones Disponibles:

print ("Posiciones Disponibles:")
print(dataset['Pos'].unique()) # 10 Posiciones.

# Selección / Filtrado de las Características Más Relevantes (Incluyendo la Columna de la Clase Objetivo ("Pos")):

caracteristicas = ['Pos', 'Blocks', 'Clr', 'Int', 'Tkl', 'TklWon', 'Press', 'PresSucc', 'Recov',
                   'PasTotDist', 'PasTotCmp', 'PasProg', 'Assists', 'CarPrgDist', 'CarTotDist',
                   'SCA', 'GCA', 'Goals', 'Shots', 'SoT', 'G/Sh', 'PKatt', 'Carries']

# Dataset Final (Filtrado con las Carcaterísticas Más Relevantes):

dataset_filtrado = dataset[caracteristicas]
print("\nDataset Filtrado:")
dataset_filtrado.head()

Posiciones Disponibles:
['DF' 'MF' 'FW' 'MFFW' 'FWMF' 'GK' 'DFMF' 'FWDF' 'MFDF' 'DFFW' 'GKMF']

Dataset Filtrado:


,Pos,Blocks,Clr,Int,Tkl,TklWon,Press,PresSucc,Recov,PasTotDist,...,CarPrgDist,CarTotDist,SCA,GCA,Goals,Shots,SoT,G/Sh,PKatt,Carries
0,DF,2.69,2.19,1.75,2.16,1.16,13.6,3.53,5.53,574.1,...,121.7,199.4,1.19,0.16,0.00,0.41,0.06,0.00,0.0,33.9
1,DF,1.87,3.20,3.11,1.87,1.39,13.6,4.89,6.77,835.8,...,115.5,204.7,0.63,0.03,0.06,0.54,0.18,0.11,0.0,35.7
2,MF,0.99,0.55,1.86,2.01,1.24,23.4,6.53,8.76,1033.3,...,106.3,246.5,1.46,0.04,0.04,0.66,0.18,0.06,0.0,53.5
3,MF,1.68,0.34,2.56,3.57,2.23,28.0,7.90,8.87,780.8,...,86.4,171.9,2.01,0.15,0.00,0.91,0.21,0.00,0.0,45.7
4,FW,2.00,0.00,0.00,0.00,0.00,28.0,2.00,4.00,48.0,...,18.0,118.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,18.0


In [4]:
# Transformación de la Variable Categórica ("Pos") a Numérica:

label_encoder = LabelEncoder() # Texto -> Números.
dataset_filtrado.loc[:,'Pos'] = label_encoder.fit_transform(dataset_filtrado['Pos']) # Posiciones (10) Convertidas a Números.

print ("Asignación de Posiciones:")
for i in range(len(label_encoder.classes_)): # Recorre la Lista 10 Veces = Número de Posiciones.
    posicion = label_encoder.classes_[i] # Guarda el Nombre de la Posición en i (Ejemplos: DF = 0; DFFFW = 1; DFMF = 2; etc).
    print (f"Posición '{posicion}' se ha convertido en el número {i}.") # Muestra que Número Corresponde a Cada Posición.

Asignación de Posiciones:
Posición 'DF' se ha convertido en el número 0.
Posición 'DFFW' se ha convertido en el número 1.
Posición 'DFMF' se ha convertido en el número 2.
Posición 'FW' se ha convertido en el número 3.
Posición 'FWDF' se ha convertido en el número 4.
Posición 'FWMF' se ha convertido en el número 5.
Posición 'GK' se ha convertido en el número 6.
Posición 'GKMF' se ha convertido en el número 7.
Posición 'MF' se ha convertido en el número 8.
Posición 'MFDF' se ha convertido en el número 9.
Posición 'MFFW' se ha convertido en el número 10.


# 3. **Separación de Datos, Normalización y División en Train y Tests**

In [5]:
# Separación de la Entrada (X) y la Salida (Y):

X = dataset_filtrado.drop ('Pos', axis=1).values # Selección de las Variables de Entrada (Todas Excepto "Pos").
Y = dataset_filtrado['Pos'].values # Selección de la Variable de Salida ("Pos").

print ("Variables Estadísticas del Jugador:")
print (X[:3]) # Muestra las Características de 3 Jugadores (Ejemplo).

print ("\nPosición del Jugador Codificada (0 al 10):")
print (Y[:3]) # Muestra las Posiciones de los 3 Jugadores (Ejemplo).

# Normalización de los Datos de Entrada (Entre 0 y 1):

scaler = MinMaxScaler() # Datos -> Datos Normalizados (Entre 0 y 1).
X = scaler.fit_transform(X)

print("\nVariables Estadísticas del Jugador (Normalizadas):")
print(X[:3]) # Muestra las Características Normalizadas de los 3 jugadores.

# División en Train (80%) y Test (20%):

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print ("\nComprobación División (Train y Test):")
print (f"80% Train: {len(X_train)}") # Comprobación del Número de Jugadores para Entrenar el Modelo (Jugadores Totales = 2.921).
print (f"20% Test: {len(X_test)}") # Comprobación del Número de Jugadores para Validar el Modelo (Jugadores Totales = 2.921).

Variables Estadísticas del Jugador:
[[2.6900e+00 2.1900e+00 1.7500e+00 2.1600e+00 1.1600e+00 1.3600e+01
  3.5300e+00 5.5300e+00 5.7410e+02 3.4000e+01 2.9400e+00 6.0000e-02
  1.2170e+02 1.9940e+02 1.1900e+00 1.6000e-01 0.0000e+00 4.1000e-01
  6.0000e-02 0.0000e+00 0.0000e+00 3.3900e+01]
 [1.8700e+00 3.2000e+00 3.1100e+00 1.8700e+00 1.3900e+00 1.3600e+01
  4.8900e+00 6.7700e+00 8.3580e+02 3.8700e+01 2.7200e+00 0.0000e+00
  1.1550e+02 2.0470e+02 6.3000e-01 3.0000e-02 6.0000e-02 5.4000e-01
  1.8000e-01 1.1000e-01 0.0000e+00 3.5700e+01]
 [9.9000e-01 5.5000e-01 1.8600e+00 2.0100e+00 1.2400e+00 2.3400e+01
  6.5300e+00 8.7600e+00 1.0333e+03 5.5900e+01 2.9600e+00 0.0000e+00
  1.0630e+02 2.4650e+02 1.4600e+00 4.0000e-02 4.0000e-02 6.6000e-01
  1.8000e-01 6.0000e-02 0.0000e+00 5.3500e+01]]

Posición del Jugador Codificada (0 al 10):
[0 0 8]

Variables Estadísticas del Jugador (Normalizadas):
[[0.1345     0.1095     0.0875     0.108      0.116      0.136
  0.11766667 0.13825    0.17719136 0.2125  

# 4. **Conversión de Datos a Tensores (Pytorch)**

In [6]:
# Conversión de la Entrada a Decimal (Tipo de Dato = Decimal):

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

# Conversión de la Salida a Entero (Tipo de Dato = Entero):

Y_train = Y_train.astype(np.int64) # int64 = long
Y_test = Y_test.astype(np.int64) # int64 = long

X_train_tensor = torch.tensor(X_train, dtype=torch.float32) # Entrada -> float32 (Decimal).
Y_train_tensor = torch.tensor(Y_train, dtype=torch.long) # Salida -> long (Entero).

X_test_tensor = torch.tensor(X_test, dtype=torch.float32) # Entrada -> float32 (Decimal).
Y_test_tensor = torch.tensor(Y_test, dtype=torch.long) # Salida -> long (Entero).

# 5. **Creación del Dataset y del DataLoader (Pytorch)**

In [7]:
# Creación del Dataset con los Tensores (Entrada + Salida):

Train_dataset = TensorDataset(X_train_tensor, Y_train_tensor) # Junta la Entrada y la Salida (Dataset de Entrenamiento).
Test_dataset = TensorDataset(X_test_tensor, Y_test_tensor) # Junta la Entrada y la Salidas (Dataset de Test).

# Creación del DataLoader (Agrupa los Datos en "Batches" (Grupos)):

Train_Loader = DataLoader(Train_dataset, batch_size=32, shuffle=True) # Batches Aleatorios (de 32) en cada Vuelta -> Red Aprende Mejor.
Test_Loader = DataLoader(Test_dataset, batch_size=32, shuffle=False) # Batches Fijos (de 32) en cada Vuelta -> Red Evalua Correctamente.

# 6. **Modelo 1: Lineal (0 Capas Ocultas)**

* Entrenado mediante el  **Descenso de Gradiente Estocástico**.

In [8]:
# Definición de las Dimensiones de Entrada y Salida:

input_dim = X_train_tensor.shape[1] # Datos de Entrada (22).
output_dim = len(np.unique(Y)) # Salidas Posibles (10).

class ModeloLineal(nn.Module):

  def __init__(self):
      super(ModeloLineal, self).__init__()
      self.linear = nn.Linear(input_dim, output_dim) # Capa Lineal: Entrada -> Salida (XW + b). No hay Capas Ocultas.

  def forward (self, x): # Definición del Flujo de Datos del Modelo.
       return self.linear(x) # Aplicación de la "Capa Lineal".

modelo_lineal = ModeloLineal() # Creación del Modelo.

In [9]:
# Definición de la Función de Coste y el Optimizador:

funcion_coste = nn.CrossEntropyLoss() # Compara las Predicciones del Modelo con los Valores Reales (Calcula el Error).
optimizador_SGD = optim.SGD(modelo_lineal.parameters(), lr=0.01) # Actualiza los Parámetros (Descenso de Gradiente). Learning Rate Controla Cuánto se Ajustan los Parámetros (Controla el Aprendizaje).

In [10]:
# Entrenamiento del Modelo:

# Fijación de Semillas Aleatorias (Garantizar Resultados Comparables entre los Diferentes Modelos)

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Reinicio del Modelo y Optimizador (Evita que el Modelo siga Aprendiendo de Ejecuciones Anteriores):

modelo_lineal = ModeloLineal() # Modelo Reiniciado.
optimizador_SGD = optim.SGD(modelo_lineal.parameters(), lr=0.01) # Optimizador Reiniciado.

# Fijación del Número de Veces que se Entrenará el Modelo:

numero_vueltas = 50

print ("Entrenamiento del Modelo:")

for epoch in range (numero_vueltas):
    modelo_lineal.train()
    error_acumulado = 0.0 # Acumulador del Error de cada Vuelta (Acumula el Error de Todos los Batches en cada una de las Vueltas).

    for inputs, labels in Train_Loader: # Ejecución por cada Batch de Train.

        # 1. Resetear los Gradiendes Calculados en Entrenamientos Anteriores:
        optimizador_SGD.zero_grad()

        # 2. Forward (Modelo Realiza Predicciones):
        outputs = modelo_lineal(inputs)

        # 3. Cálculo del Error:
        error = funcion_coste(outputs, labels)

        # 4. Backward / Retropropagación (Cálculo de Gradientes para Ajustar los Parámetros):
        error.backward()

        # 5. Optimizar (Actualización de los Parámetros):
        optimizador_SGD.step()

        # 6. Acumulación del Error (Sumatorio del Error de cada Batch = Error Total de cada Vuelta):
        error_acumulado += error.item()

    print(f"Vuelta {epoch+1}/{numero_vueltas}, Error: {error_acumulado/len(Train_Loader):.4f}")

Entrenamiento del Modelo:
Vuelta 1/50, Error: 2.3012
Vuelta 2/50, Error: 2.2224
Vuelta 3/50, Error: 2.1583
Vuelta 4/50, Error: 2.1066
Vuelta 5/50, Error: 2.0649
Vuelta 6/50, Error: 2.0312
Vuelta 7/50, Error: 2.0038
Vuelta 8/50, Error: 1.9812
Vuelta 9/50, Error: 1.9623
Vuelta 10/50, Error: 1.9462
Vuelta 11/50, Error: 1.9323
Vuelta 12/50, Error: 1.9201
Vuelta 13/50, Error: 1.9092
Vuelta 14/50, Error: 1.8995
Vuelta 15/50, Error: 1.8907
Vuelta 16/50, Error: 1.8826
Vuelta 17/50, Error: 1.8752
Vuelta 18/50, Error: 1.8683
Vuelta 19/50, Error: 1.8618
Vuelta 20/50, Error: 1.8558
Vuelta 21/50, Error: 1.8501
Vuelta 22/50, Error: 1.8447
Vuelta 23/50, Error: 1.8395
Vuelta 24/50, Error: 1.8347
Vuelta 25/50, Error: 1.8300
Vuelta 26/50, Error: 1.8255
Vuelta 27/50, Error: 1.8212
Vuelta 28/50, Error: 1.8170
Vuelta 29/50, Error: 1.8130
Vuelta 30/50, Error: 1.8091
Vuelta 31/50, Error: 1.8053
Vuelta 32/50, Error: 1.8016
Vuelta 33/50, Error: 1.7981
Vuelta 34/50, Error: 1.7946
Vuelta 35/50, Error: 1.7912
Vue

In [11]:
# Evaluación del Modelo:

modelo_lineal.eval()

# Creación de Contadores (Predicciones Correctas y Jugadores Totales Evaluados):

numero_aciertos = 0
numero_jugadores_evaluados = 0

with torch.no_grad():

     for inputs, labels in Test_Loader: # Ejecución por cada Batch de Test.

         # 1. Forward (Modelo Realiza Predicciones):
         outputs = modelo_lineal(inputs)

         # 2. Selección de la Clase Predicha (Mayor Probabilidad):
         predicciones = torch.argmax(outputs, dim=1)

         # 3. Cálculo del Número de Predicciones Correctas (Predicciones = Etiquetas Reales):
         numero_aciertos += (predicciones == labels).sum().item()

         # 4. Cálculo del Número Total de Jugadores Evaluados:
         numero_jugadores_evaluados += labels.size(0)

# 5. Calculo del Accuracy (% Predicciones Correctas):
accuracy = 100 * numero_aciertos / numero_jugadores_evaluados

print(f"Precisión del Modelo: {accuracy:.2f}%")

Precisión del Modelo: 32.65%


# 7. **Modelo 2: Capas Ocultas (3) + RELU**

* Entrenado mediante el  **Descenso de Gradiente Estocástico**.

In [12]:
# Definición de las Dimensiones de Entrada y Salida:

input_dim = X_train_tensor.shape[1] # Datos de Entrada (22).
output_dim = len(np.unique(Y)) # Salidas Posibles (10).

class ModeloCapasOcultasRELU(nn.Module):

  def __init__(self):
      super(ModeloCapasOcultasRELU, self).__init__()

      self.capa_oculta1 = nn.Linear(input_dim, 128) # Capa Oculta (128 Neuronas).
      self.funcion_activacion1 = nn.ReLU() # RELU.

      self.capa_oculta2 = nn.Linear(128, 64) # Capa Oculta (64 Neuronas).
      self.funcion_activacion2 = nn.ReLU() # RELU.

      self.capa_oculta3 = nn.Linear(64, 32) # Capa Oculta (32 Neuronas).
      self.funcion_activacion3 = nn.ReLU() # RELU.

      self.capa_salida = nn.Linear(32, output_dim) # Capa de Salida.

  def forward (self, x): # Definición del Flujo de Datos del Modelo.

      # 1. Paso por la Primera Capa Oculta y Aplicación de la RELU:

      x = self.capa_oculta1(x)
      x = self.funcion_activacion1(x)

      # 2. Paso por la Segunda Capa Oculta y Aplicación de la RELU:

      x = self.capa_oculta2(x)
      x = self.funcion_activacion2(x)

      # 3. Paso por la Tercera Capa Oculta y Aplicación de la RELU:

      x = self.capa_oculta3(x)
      x = self.funcion_activacion3(x)

      # 4. Paso por la Capa de Salida:

      x = self.capa_salida(x)
      return x

modelo_Capas_Ocultas_RELU = ModeloCapasOcultasRELU() # Creación del Modelo.

In [13]:
# Definición de la Función de Coste y el Optimizador:

funcion_coste = nn.CrossEntropyLoss() # Compara las Predicciones del Modelo con los Valores Reales (Calcula el Error).
optimizador_SGD = optim.SGD(modelo_Capas_Ocultas_RELU.parameters(), lr=0.01) # Actualiza los Parámetros (Descenso de Gradiente). Learning Rate Controla Cuánto se Ajustan los Parámetros (Controla el Aprendizaje).

In [14]:
# Entrenamiento del Modelo:

# Fijación de Semillas Aleatorias (Garantizar Resultados Comparables entre los Diferentes Modelos)

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Reinicio del Modelo y Optimizador (Evita que el Modelo siga Aprendiendo de Ejecuciones Anteriores):

modelo_Capas_Ocultas_RELU = ModeloCapasOcultasRELU() # Modelo Reiniciado.
optimizador_SGD = optim.SGD(modelo_Capas_Ocultas_RELU.parameters(), lr=0.01) # Optimizador Reiniciado.

# Fijación del Número de Veces que se Entrenará el Modelo:

numero_vueltas = 100

print ("Entrenamiento del Modelo:")

for epoch in range (numero_vueltas):
    modelo_Capas_Ocultas_RELU.train()
    error_acumulado = 0.0 # Acumulador del Error de cada Vuelta (Acumula el Error de Todos los Batches en cada una de las Vueltas).

    for inputs, labels in Train_Loader: # Ejecución por cada Batch de Train.

        # 1. Resetear los Gradientes Calculados en Entrenamientos Anteriores:
        optimizador_SGD.zero_grad()

        # 2. Forward (Modelo Realiza Predicciones):
        outputs = modelo_Capas_Ocultas_RELU(inputs)

        # 3. Cálculo del Error:
        error = funcion_coste(outputs, labels)

        # 4. Backward / Retropropagación (Cálculo de Gradientes para Ajustar los Parámetros):
        error.backward()

        # 5. Optimizar (Actualización de los Parámetros):
        optimizador_SGD.step()

        # 6. Acumulación del Error (Sumatorio del Error de cada Batch = Error Total de cada Vuelta):
        error_acumulado += error.item()

    print(f"Vuelta {epoch+1}/{numero_vueltas}, Error: {error_acumulado/len(Train_Loader):.4f}")

Entrenamiento del Modelo:
Vuelta 1/100, Error: 2.3569
Vuelta 2/100, Error: 2.2844
Vuelta 3/100, Error: 2.2193
Vuelta 4/100, Error: 2.1599
Vuelta 5/100, Error: 2.1058
Vuelta 6/100, Error: 2.0573
Vuelta 7/100, Error: 2.0157
Vuelta 8/100, Error: 1.9820
Vuelta 9/100, Error: 1.9564
Vuelta 10/100, Error: 1.9376
Vuelta 11/100, Error: 1.9239
Vuelta 12/100, Error: 1.9133
Vuelta 13/100, Error: 1.9053
Vuelta 14/100, Error: 1.8990
Vuelta 15/100, Error: 1.8941
Vuelta 16/100, Error: 1.8900
Vuelta 17/100, Error: 1.8866
Vuelta 18/100, Error: 1.8836
Vuelta 19/100, Error: 1.8810
Vuelta 20/100, Error: 1.8785
Vuelta 21/100, Error: 1.8764
Vuelta 22/100, Error: 1.8741
Vuelta 23/100, Error: 1.8718
Vuelta 24/100, Error: 1.8693
Vuelta 25/100, Error: 1.8669
Vuelta 26/100, Error: 1.8642
Vuelta 27/100, Error: 1.8613
Vuelta 28/100, Error: 1.8579
Vuelta 29/100, Error: 1.8545
Vuelta 30/100, Error: 1.8505
Vuelta 31/100, Error: 1.8459
Vuelta 32/100, Error: 1.8407
Vuelta 33/100, Error: 1.8348
Vuelta 34/100, Error: 1.82

In [15]:
# Evaluación del Modelo:

modelo_Capas_Ocultas_RELU.eval()

# Creación de Contadores (Predicciones Correctas y Jugadores Totales Evaluados):

numero_aciertos = 0
numero_jugadores_evaluados = 0

with torch.no_grad():

     for inputs, labels in Test_Loader: # Ejecución por cada Batch de Test.

         # 1. Forward (Modelo Realiza Predicciones):
         outputs = modelo_Capas_Ocultas_RELU(inputs)

         # 2. Selección de la Clase Predicha (Mayor Probabilidad):
         predicciones = torch.argmax(outputs, dim=1)

         # 3. Cálculo del Número de Predicciones Correctas (Predicciones = Etiquetas Reales):
         numero_aciertos += (predicciones == labels).sum().item()

         # 4. Cálculo del Número Total de Jugadores Evaluados:
         numero_jugadores_evaluados += labels.size(0)

# 5. Calculo del Accuracy (% Predicciones Correctas):
accuracy = 100 * numero_aciertos / numero_jugadores_evaluados

print(f"Precisión del Modelo: {accuracy:.2f}%")

Precisión del Modelo: 65.47%


# 7. **Modelo 3: Capas Ocultas (3) + Tangente Hiperbólica**

* Entrenado mediante el  **Descenso de Gradiente Estocástico**.

In [16]:
# Definición de las Dimensiones de Entrada y Salida:

input_dim = X_train_tensor.shape[1] # Datos de Entrada (22).
output_dim = len(np.unique(Y)) # Salidas Posibles (10).

class ModeloCapasOcultasTanh(nn.Module):

  def __init__(self):
      super(ModeloCapasOcultasTanh, self).__init__()

      self.capa_oculta1 = nn.Linear(input_dim, 128) # Capa Oculta (128 Neuronas).
      self.funcion_activacion1 = nn.Tanh() # Tanh.

      self.capa_oculta2 = nn.Linear(128, 64) # Capa Oculta (64 Neuronas).
      self.funcion_activacion2 = nn.Tanh() # Tanh.

      self.capa_oculta3 = nn.Linear(64, 32) # Capa Oculta (32 Neuronas).
      self.funcion_activacion3 = nn.Tanh() # Tanh.

      self.capa_salida = nn.Linear(32, output_dim) # Capa de Salida.

  def forward (self, x): # Definición del Flujo de Datos del Modelo.

      # 1. Paso por la Primera Capa Oculta y Aplicación de la Tanh:

      x = self.capa_oculta1(x)
      x = self.funcion_activacion1(x)

      # 2. Paso por la Segunda Capa Oculta y Aplicación de la Tanh:

      x = self.capa_oculta2(x)
      x = self.funcion_activacion2(x)

      # 3. Paso por la Tercera Capa Oculta y Aplicación de la Tanh:

      x = self.capa_oculta3(x)
      x = self.funcion_activacion3(x)

      # 4. Paso por la Capa de Salida:

      x = self.capa_salida(x)
      return x

modelo_Capas_Ocultas_Tanh = ModeloCapasOcultasTanh() # Creación del Modelo.

In [17]:
# Definición de la Función de Coste y el Optimizador:

funcion_coste = nn.CrossEntropyLoss() # Compara las Predicciones del Modelo con los Valores Reales (Calcula el Error).
optimizador_SGD = optim.SGD(modelo_Capas_Ocultas_Tanh.parameters(), lr=0.005) # Actualiza los Parámetros (Descenso de Gradiente). Learning Rate Controla Cuánto se Ajustan los Parámetros (Controla el Aprendizaje).

In [18]:
# Entrenamiento del Modelo:

# Fijación de Semillas Aleatorias (Garantizar Resultados Comparables entre los Diferentes Modelos)

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Reinicio del Modelo y Optimizador (Evita que el Modelo siga Aprendiendo de Ejecuciones Anteriores):

modelo_Capas_Ocultas_Tanh = ModeloCapasOcultasTanh() # Modelo Reiniciado.
optimizador_SGD = optim.SGD(modelo_Capas_Ocultas_Tanh.parameters(), lr=0.005) # Optimizador Reiniciado.

# Fijación del Número de Veces que se Entrenará el Modelo:

numero_vueltas = 100

print ("Entrenamiento del Modelo:")

for epoch in range (numero_vueltas):
    modelo_Capas_Ocultas_Tanh.train()
    error_acumulado = 0.0 # Acumulador del Error de cada Vuelta (Acumula el Error de Todos los Batches en cada una de las Vueltas).

    for inputs, labels in Train_Loader: # Ejecución por cada Batch de Train.

        # 1. Resetear los Gradientes Calculados en Entrenamientos Anteriores:
        optimizador_SGD.zero_grad()

        # 2. Forward (Modelo Realiza Predicciones):
        outputs = modelo_Capas_Ocultas_Tanh(inputs)

        # 3. Cálculo del Error:
        error = funcion_coste(outputs, labels)

        # 4. Backward / Retropropagación (Cálculo de Gradientes para Ajustar los Parámetros):
        error.backward()

        # 5. Optimizar (Actualización de los Parámetros):
        optimizador_SGD.step()

        # 6. Acumulación del Error (Sumatorio del Error de cada Batch = Error Total de cada Vuelta):
        error_acumulado += error.item()

    print(f"Vuelta {epoch+1}/{numero_vueltas}, Error: {error_acumulado/len(Train_Loader):.4f}")

Entrenamiento del Modelo:
Vuelta 1/100, Error: 2.3539
Vuelta 2/100, Error: 2.2906
Vuelta 3/100, Error: 2.2300
Vuelta 4/100, Error: 2.1712
Vuelta 5/100, Error: 2.1150
Vuelta 6/100, Error: 2.0637
Vuelta 7/100, Error: 2.0200
Vuelta 8/100, Error: 1.9856
Vuelta 9/100, Error: 1.9599
Vuelta 10/100, Error: 1.9410
Vuelta 11/100, Error: 1.9271
Vuelta 12/100, Error: 1.9162
Vuelta 13/100, Error: 1.9077
Vuelta 14/100, Error: 1.9007
Vuelta 15/100, Error: 1.8949
Vuelta 16/100, Error: 1.8900
Vuelta 17/100, Error: 1.8857
Vuelta 18/100, Error: 1.8819
Vuelta 19/100, Error: 1.8784
Vuelta 20/100, Error: 1.8752
Vuelta 21/100, Error: 1.8723
Vuelta 22/100, Error: 1.8694
Vuelta 23/100, Error: 1.8666
Vuelta 24/100, Error: 1.8638
Vuelta 25/100, Error: 1.8611
Vuelta 26/100, Error: 1.8583
Vuelta 27/100, Error: 1.8554
Vuelta 28/100, Error: 1.8524
Vuelta 29/100, Error: 1.8493
Vuelta 30/100, Error: 1.8460
Vuelta 31/100, Error: 1.8425
Vuelta 32/100, Error: 1.8388
Vuelta 33/100, Error: 1.8348
Vuelta 34/100, Error: 1.83

In [19]:
# Evaluación del Modelo:

modelo_Capas_Ocultas_Tanh.eval()

# Creación de Contadores (Predicciones Correctas y Jugadores Totales Evaluados):

numero_aciertos = 0
numero_jugadores_evaluados = 0

with torch.no_grad():

     for inputs, labels in Test_Loader: # Ejecución por cada Batch de Test.

         # 1. Forward (Modelo Realiza Predicciones):
         outputs = modelo_Capas_Ocultas_Tanh(inputs)

         # 2. Selección de la Clase Predicha (Mayor Probabilidad):
         predicciones = torch.argmax(outputs, dim=1)

         # 3. Cálculo del Número de Predicciones Correctas (Predicciones = Etiquetas Reales):
         numero_aciertos += (predicciones == labels).sum().item()

         # 4. Cálculo del Número Total de Jugadores Evaluados:
         numero_jugadores_evaluados += labels.size(0)

# 5. Calculo del Accuracy (% Predicciones Correctas):
accuracy = 100 * numero_aciertos / numero_jugadores_evaluados

print(f"Precisión del Modelo: {accuracy:.2f}%")

Precisión del Modelo: 60.17%


# 8. **Modelo 4: Capas Ocultas (3) + RELU + BatchNormalization**

* Entrenado mediante el  **Descenso de Gradiente Estocástico**.

In [20]:
# Definición de las Dimensiones de Entrada y Salida:

input_dim = X_train_tensor.shape[1] # Datos de Entrada (22).
output_dim = len(np.unique(Y)) # Salidas Posibles (10).

class ModeloBatchNormalization(nn.Module):

  def __init__(self):
      super(ModeloBatchNormalization, self).__init__()

      self.capa_oculta1 = nn.Linear(input_dim, 128) # Capa Oculta (128 Neuronas).
      self.batch_normalization1 = nn.BatchNorm1d(128) # BatchNormalization.
      self.funcion_activacion1 = nn.ReLU() # RELU.

      self.capa_oculta2 = nn.Linear(128, 64) # Capa Oculta (64 Neuronas).
      self.batch_normalization2 = nn.BatchNorm1d(64) # BatchNormalization.
      self.funcion_activacion2 = nn.ReLU() # RELU.

      self.capa_oculta3 = nn.Linear(64, 32) # Capa Oculta (32 Neuronas).
      self.batch_normalization3 = nn.BatchNorm1d(32) # BatchNormalization.
      self.funcion_activacion3 = nn.ReLU() # RELU.

      self.capa_salida = nn.Linear(32, output_dim) # Capa de Salida.

  def forward (self, x): # Definición del Flujo de Datos del Modelo.

      # 1. Paso por la Primera Capa Oculta, Normalización y Aplicación de la RELU:

      x = self.capa_oculta1(x)
      x = self.batch_normalization1(x)
      x = self.funcion_activacion1(x)

      # 2. Paso por la Segunda Capa Oculta, Normalización y Aplicación de la RELU:

      x = self.capa_oculta2(x)
      x = self.batch_normalization2(x)
      x = self.funcion_activacion2(x)

      # 3. Paso por la Tercera Capa Oculta, Normalización y Aplicación de la RELU:

      x = self.capa_oculta3(x)
      x = self.batch_normalization3(x)
      x = self.funcion_activacion3(x)

      # 4. Paso por la Capa de Salida:

      x = self.capa_salida(x)
      return x

modelo_batch_normalization = ModeloBatchNormalization() # Creación del Modelo.

In [21]:
# Definición de la Función de Coste y el Optimizador:

funcion_coste = nn.CrossEntropyLoss() # Compara las Predicciones del Modelo con los Valores Reales (Calcula el Error).
optimizador_SGD = optim.SGD(modelo_batch_normalization.parameters(), lr=0.01) # Actualiza los Parámetros (Descenso de Gradiente). Learning Rate Controla Cuánto se Ajustan los Parámetros (Controla el Aprendizaje).

In [22]:
# Entrenamiento del Modelo:

# Fijación de Semillas Aleatorias (Garantizar Resultados Comparables entre los Diferentes Modelos)

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Reinicio del Modelo y Optimizador (Evita que el Modelo siga Aprendiendo de Ejecuciones Anteriores):

modelo_batch_normalization = ModeloBatchNormalization() # Modelo Reiniciado.
optimizador_SGD = optim.SGD(modelo_batch_normalization.parameters(), lr=0.01) # Optimizador Reiniciado.

# Fijación del Número de Veces que se Entrenará el Modelo:

numero_vueltas = 100

print ("Entrenamiento del Modelo:")

for epoch in range (numero_vueltas):
    modelo_batch_normalization.train()
    error_acumulado = 0.0 # Acumulador del Error de cada Vuelta (Acumula el Error de Todos los Batches en cada una de las Vueltas).

    for inputs, labels in Train_Loader: # Ejecución por cada Batch de Train.

        # 1. Resetear los Gradientes Calculados en Entrenamientos Anteriores:
        optimizador_SGD.zero_grad()

        # 2. Forward (Modelo Realiza Predicciones):
        outputs = modelo_batch_normalization(inputs)

        # 3. Cálculo del Error:
        error = funcion_coste(outputs, labels)

        # 4. Backward / Retropropagación (Cálculo de Gradientes para Ajustar los Parámetros):
        error.backward()

        # 5. Optimizar (Actualización de los Parámetros):
        optimizador_SGD.step()

        # 6. Acumulación del Error (Sumatorio del Error de cada Batch = Error Total de cada Vuelta):
        error_acumulado += error.item()

    print(f"Vuelta {epoch+1}/{numero_vueltas}, Error: {error_acumulado/len(Train_Loader):.4f}")

Entrenamiento del Modelo:
Vuelta 1/100, Error: 1.9300
Vuelta 2/100, Error: 1.4176
Vuelta 3/100, Error: 1.2154
Vuelta 4/100, Error: 1.1090
Vuelta 5/100, Error: 1.0344
Vuelta 6/100, Error: 0.9866
Vuelta 7/100, Error: 0.9605
Vuelta 8/100, Error: 0.9466
Vuelta 9/100, Error: 0.9220
Vuelta 10/100, Error: 0.8907
Vuelta 11/100, Error: 0.8882
Vuelta 12/100, Error: 0.8560
Vuelta 13/100, Error: 0.8544
Vuelta 14/100, Error: 0.8353
Vuelta 15/100, Error: 0.8331
Vuelta 16/100, Error: 0.8299
Vuelta 17/100, Error: 0.8177
Vuelta 18/100, Error: 0.8031
Vuelta 19/100, Error: 0.7971
Vuelta 20/100, Error: 0.7814
Vuelta 21/100, Error: 0.7993
Vuelta 22/100, Error: 0.7821
Vuelta 23/100, Error: 0.7648
Vuelta 24/100, Error: 0.7582
Vuelta 25/100, Error: 0.7629
Vuelta 26/100, Error: 0.7660
Vuelta 27/100, Error: 0.7500
Vuelta 28/100, Error: 0.7239
Vuelta 29/100, Error: 0.7288
Vuelta 30/100, Error: 0.7455
Vuelta 31/100, Error: 0.7354
Vuelta 32/100, Error: 0.7138
Vuelta 33/100, Error: 0.7058
Vuelta 34/100, Error: 0.71

In [23]:
# Evaluación del Modelo:

modelo_batch_normalization.eval()

# Creación de Contadores (Predicciones Correctas y Jugadores Totales Evaluados):

numero_aciertos = 0
numero_jugadores_evaluados = 0

with torch.no_grad():

     for inputs, labels in Test_Loader: # Ejecución por cada Batch de Test.

         # 1. Forward (Modelo Realiza Predicciones):
         outputs = modelo_batch_normalization(inputs)

         # 2. Selección de la Clase Predicha (Mayor Probabilidad):
         predicciones = torch.argmax(outputs, dim=1)

         # 3. Cálculo del Número de Predicciones Correctas (Predicciones = Etiquetas Reales):
         numero_aciertos += (predicciones == labels).sum().item()

         # 4. Cálculo del Número Total de Jugadores Evaluados:
         numero_jugadores_evaluados += labels.size(0)

# 5. Calculo del Accuracy (% Predicciones Correctas):
accuracy = 100 * numero_aciertos / numero_jugadores_evaluados

print(f"Precisión del Modelo: {accuracy:.2f}%")

Precisión del Modelo: 68.89%


# 8. **Modelo 5: Capas Ocultas (3) + RELU + Dropout**

* Entrenado mediante el  **Descenso de Gradiente Estocástico**.

In [24]:
# Definición de las Dimensiones de Entrada y Salida:

input_dim = X_train_tensor.shape[1] # Datos de Entrada (22).
output_dim = len(np.unique(Y)) # Salidas Posibles (10).

class ModeloDropout(nn.Module):

  def __init__(self):
      super(ModeloDropout, self).__init__()

      self.capa_oculta1 = nn.Linear(input_dim, 128) # Capa Oculta (128 Neuronas).
      self.funcion_activacion1 = nn.ReLU() # RELU.
      # Dropout no Aplicado (Resultados Peores en la Realización de Pruebas).

      self.capa_oculta2 = nn.Linear(128, 64) # Capa Oculta (64 Neuronas).
      self.funcion_activacion2 = nn.ReLU() # RELU.
      self.dropout2 = nn.Dropout(p=0.2) # Dropout (Desactivación Aleatoria del 20% de las Neuronas).

      self.capa_oculta3 = nn.Linear(64, 32) # Capa Oculta (32 Neuronas).
      self.funcion_activacion3 = nn.ReLU() # RELU.
      self.dropout3 = nn.Dropout(p=0.2) # Dropout (Desactivación Aleatoria del 20% de las Neuronas).

      self.capa_salida = nn.Linear(32, output_dim) # Capa de Salida.

  def forward (self, x): # Definición del Flujo de Datos del Modelo.

      # 1. Paso por la Primera Capa Oculta y Aplicación de la RELU:

      x = self.capa_oculta1(x)
      x = self.funcion_activacion1(x)
      # Dropout no Aplicado (Resultados Peores en la Realización de Pruebas).

      # 2. Paso por la Segunda Capa Oculta, Aplicación de la RELU y Aplicación del Dropout:

      x = self.capa_oculta2(x)
      x = self.funcion_activacion2(x)
      x = self.dropout2(x)

      # 3. Paso por la Tercera Capa Oculta, Aplicación de la RELU y Aplicación del Dropout:

      x = self.capa_oculta3(x)
      x = self.funcion_activacion3(x)
      x = self.dropout3(x)

      # 4. Paso por la Capa de Salida:

      x = self.capa_salida(x)
      return x

modelo_dropout = ModeloDropout() # Creación del Modelo.

In [25]:
# Definición de la Función de Coste y el Optimizador:

funcion_coste = nn.CrossEntropyLoss() # Compara las Predicciones del Modelo con los Valores Reales (Calcula el Error).
optimizador_SGD = optim.SGD(modelo_dropout.parameters(), lr=0.01) # Actualiza los Parámetros (Descenso de Gradiente). Learning Rate Controla Cuánto se Ajustan los Parámetros (Controla el Aprendizaje).

In [26]:
# Entrenamiento del Modelo:

# Fijación de Semillas Aleatorias (Garantizar Resultados Comparables entre los Diferentes Modelos)

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Reinicio del Modelo y Optimizador (Evita que el Modelo siga Aprendiendo de Ejecuciones Anteriores):

modelo_dropout = ModeloDropout() # Modelo Reiniciado.
optimizador_SGD = optim.SGD(modelo_dropout.parameters(), lr=0.01) # Optimizador Reiniciado.

# Fijación del Número de Veces que se Entrenará el Modelo:

numero_vueltas = 100

print ("Entrenamiento del Modelo:")

for epoch in range (numero_vueltas):
    modelo_dropout.train()
    error_acumulado = 0.0 # Acumulador del Error de cada Vuelta (Acumula el Error de Todos los Batches en cada una de las Vueltas).

    for inputs, labels in Train_Loader: # Ejecución por cada Batch de Train.

        # 1. Resetear los Gradientes Calculados en Entrenamientos Anteriores:
        optimizador_SGD.zero_grad()

        # 2. Forward (Modelo Realiza Predicciones):
        outputs = modelo_dropout(inputs)

        # 3. Cálculo del Error:
        error = funcion_coste(outputs, labels)

        # 4. Backward / Retropropagación (Cálculo de Gradientes para Ajustar los Parámetros):
        error.backward()

        # 5. Optimizar (Actualización de los Parámetros):
        optimizador_SGD.step()

        # 6. Acumulación del Error (Sumatorio del Error de cada Batch = Error Total de cada Vuelta):
        error_acumulado += error.item()

    print(f"Vuelta {epoch+1}/{numero_vueltas}, Error: {error_acumulado/len(Train_Loader):.4f}")

Entrenamiento del Modelo:
Vuelta 1/100, Error: 2.3577
Vuelta 2/100, Error: 2.2842
Vuelta 3/100, Error: 2.2199
Vuelta 4/100, Error: 2.1613
Vuelta 5/100, Error: 2.1088
Vuelta 6/100, Error: 2.0611
Vuelta 7/100, Error: 2.0208
Vuelta 8/100, Error: 1.9889
Vuelta 9/100, Error: 1.9692
Vuelta 10/100, Error: 1.9498
Vuelta 11/100, Error: 1.9376
Vuelta 12/100, Error: 1.9252
Vuelta 13/100, Error: 1.9180
Vuelta 14/100, Error: 1.9106
Vuelta 15/100, Error: 1.9115
Vuelta 16/100, Error: 1.9028
Vuelta 17/100, Error: 1.9008
Vuelta 18/100, Error: 1.9002
Vuelta 19/100, Error: 1.8999
Vuelta 20/100, Error: 1.8897
Vuelta 21/100, Error: 1.8934
Vuelta 22/100, Error: 1.8893
Vuelta 23/100, Error: 1.8861
Vuelta 24/100, Error: 1.8866
Vuelta 25/100, Error: 1.8915
Vuelta 26/100, Error: 1.8799
Vuelta 27/100, Error: 1.8819
Vuelta 28/100, Error: 1.8821
Vuelta 29/100, Error: 1.8720
Vuelta 30/100, Error: 1.8807
Vuelta 31/100, Error: 1.8692
Vuelta 32/100, Error: 1.8640
Vuelta 33/100, Error: 1.8613
Vuelta 34/100, Error: 1.86

In [27]:
# Evaluación del Modelo:

modelo_dropout.eval()

# Creación de Contadores (Predicciones Correctas y Jugadores Totales Evaluados):

numero_aciertos = 0
numero_jugadores_evaluados = 0

with torch.no_grad():

     for inputs, labels in Test_Loader: # Ejecución por cada Batch de Test.

         # 1. Forward (Modelo Realiza Predicciones):
         outputs = modelo_dropout(inputs)

         # 2. Selección de la Clase Predicha (Mayor Probabilidad):
         predicciones = torch.argmax(outputs, dim=1)

         # 3. Cálculo del Número de Predicciones Correctas (Predicciones = Etiquetas Reales):
         numero_aciertos += (predicciones == labels).sum().item()

         # 4. Cálculo del Número Total de Jugadores Evaluados:
         numero_jugadores_evaluados += labels.size(0)

# 5. Calculo del Accuracy (% Predicciones Correctas):
accuracy = 100 * numero_aciertos / numero_jugadores_evaluados

print(f"Precisión del Modelo: {accuracy:.2f}%")

Precisión del Modelo: 62.74%


# 8. **Modelo 6: Capas Ocultas (3) + RELU + Momentum**

* Entrenado mediante el  **Descenso de Gradiente Estocástico**.

In [28]:
# Definición de las Dimensiones de Entrada y Salida:

input_dim = X_train_tensor.shape[1] # Datos de Entrada (22).
output_dim = len(np.unique(Y)) # Salidas Posibles (10).

class ModeloRELUMomentum(nn.Module):

  def __init__(self):
      super(ModeloRELUMomentum, self).__init__()

      self.capa_oculta1 = nn.Linear(input_dim, 128) # Capa Oculta (128 Neuronas).
      self.funcion_activacion1 = nn.ReLU() # RELU.

      self.capa_oculta2 = nn.Linear(128, 64) # Capa Oculta (64 Neuronas).
      self.funcion_activacion2 = nn.ReLU() # RELU.

      self.capa_oculta3 = nn.Linear(64, 32) # Capa Oculta (32 Neuronas).
      self.funcion_activacion3 = nn.ReLU() # RELU.

      self.capa_salida = nn.Linear(32, output_dim) # Capa de Salida.

  def forward (self, x): # Definición del Flujo de Datos del Modelo.

      # 1. Paso por la Primera Capa Oculta y Aplicación de la RELU:

      x = self.capa_oculta1(x)
      x = self.funcion_activacion1(x)

      # 2. Paso por la Segunda Capa Oculta y Aplicación de la RELU:

      x = self.capa_oculta2(x)
      x = self.funcion_activacion2(x)

      # 3. Paso por la Tercera Capa Oculta y Aplicación de la RELU:

      x = self.capa_oculta3(x)
      x = self.funcion_activacion3(x)

      # 4. Paso por la Capa de Salida:

      x = self.capa_salida(x)
      return x

modelo_Capas_Ocultas_RELU_Momentum = ModeloRELUMomentum() # Creación del Modelo.

In [29]:
# Definición de la Función de Coste y el Optimizador:

funcion_coste = nn.CrossEntropyLoss() # Compara las Predicciones del Modelo con los Valores Reales (Calcula el Error).
optimizador_SGD = optim.SGD(modelo_Capas_Ocultas_RELU_Momentum.parameters(), lr=0.01, momentum=0.9) # Actualiza los Parámetros (Descenso de Gradiente). Learning Rate Controla Cuánto se Ajustan los Parámetros (Controla el Aprendizaje). Momentum Acelera el Aprendizaje y Evita Oscilaciones.

In [30]:
# Entrenamiento del Modelo:

# Fijación de Semillas Aleatorias (Garantizar Resultados Comparables entre los Diferentes Modelos)

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Reinicio del Modelo y Optimizador (Evita que el Modelo siga Aprendiendo de Ejecuciones Anteriores):

modelo_Capas_Ocultas_RELU_Momentum = ModeloRELUMomentum() # Modelo Reiniciado.
optimizador_SGD = optim.SGD(modelo_Capas_Ocultas_RELU_Momentum.parameters(), lr=0.01, momentum=0.9) # Optimizador Reiniciado.

# Fijación del Número de Veces que se Entrenará el Modelo:

numero_vueltas = 100

print ("Entrenamiento del Modelo:")

for epoch in range (numero_vueltas):
    modelo_Capas_Ocultas_RELU_Momentum.train()
    error_acumulado = 0.0 # Acumulador del Error de cada Vuelta (Acumula el Error de Todos los Batches en cada una de las Vueltas).

    for inputs, labels in Train_Loader: # Ejecución por cada Batch de Train.

        # 1. Resetear los Gradientes Calculados en Entrenamientos Anteriores:
        optimizador_SGD.zero_grad()

        # 2. Forward (Modelo Realiza Predicciones):
        outputs = modelo_Capas_Ocultas_RELU_Momentum(inputs)

        # 3. Cálculo del Error:
        error = funcion_coste(outputs, labels)

        # 4. Backward / Retropropagación (Cálculo de Gradientes para Ajustar los Parámetros):
        error.backward()

        # 5. Optimizar (Actualización de los Parámetros):
        optimizador_SGD.step()

        # 6. Acumulación del Error (Sumatorio del Error de cada Batch = Error Total de cada Vuelta):
        error_acumulado += error.item()

    print(f"Vuelta {epoch+1}/{numero_vueltas}, Error: {error_acumulado/len(Train_Loader):.4f}")

Entrenamiento del Modelo:
Vuelta 1/100, Error: 2.1674
Vuelta 2/100, Error: 1.9180
Vuelta 3/100, Error: 1.8832
Vuelta 4/100, Error: 1.8568
Vuelta 5/100, Error: 1.7737
Vuelta 6/100, Error: 1.5757
Vuelta 7/100, Error: 1.4271
Vuelta 8/100, Error: 1.3332
Vuelta 9/100, Error: 1.2742
Vuelta 10/100, Error: 1.2093
Vuelta 11/100, Error: 1.1204
Vuelta 12/100, Error: 1.0804
Vuelta 13/100, Error: 1.0578
Vuelta 14/100, Error: 1.0463
Vuelta 15/100, Error: 1.0154
Vuelta 16/100, Error: 0.9941
Vuelta 17/100, Error: 0.9971
Vuelta 18/100, Error: 0.9953
Vuelta 19/100, Error: 0.9937
Vuelta 20/100, Error: 0.9789
Vuelta 21/100, Error: 0.9620
Vuelta 22/100, Error: 0.9618
Vuelta 23/100, Error: 0.9550
Vuelta 24/100, Error: 0.9453
Vuelta 25/100, Error: 0.9300
Vuelta 26/100, Error: 0.9260
Vuelta 27/100, Error: 0.9314
Vuelta 28/100, Error: 0.9148
Vuelta 29/100, Error: 0.9312
Vuelta 30/100, Error: 0.9064
Vuelta 31/100, Error: 0.9148
Vuelta 32/100, Error: 0.9039
Vuelta 33/100, Error: 0.9109
Vuelta 34/100, Error: 0.90

In [31]:
# Evaluación del Modelo:

modelo_Capas_Ocultas_RELU_Momentum.eval()

# Creación de Contadores (Predicciones Correctas y Jugadores Totales Evaluados):

numero_aciertos = 0
numero_jugadores_evaluados = 0

with torch.no_grad():

     for inputs, labels in Test_Loader: # Ejecución por cada Batch de Test.

         # 1. Forward (Modelo Realiza Predicciones):
         outputs = modelo_Capas_Ocultas_RELU_Momentum(inputs)

         # 2. Selección de la Clase Predicha (Mayor Probabilidad):
         predicciones = torch.argmax(outputs, dim=1)

         # 3. Cálculo del Número de Predicciones Correctas (Predicciones = Etiquetas Reales):
         numero_aciertos += (predicciones == labels).sum().item()

         # 4. Cálculo del Número Total de Jugadores Evaluados:
         numero_jugadores_evaluados += labels.size(0)

# 5. Calculo del Accuracy (% Predicciones Correctas):
accuracy = 100 * numero_aciertos / numero_jugadores_evaluados

print(f"Precisión del Modelo: {accuracy:.2f}%")

Precisión del Modelo: 67.86%


# 8. **Modelo 4: Capas Ocultas (3) + RELU + BatchNormalization + Momentum**

* Entrenado mediante el  **Descenso de Gradiente Estocástico**.

In [32]:
# Definición de las Dimensiones de Entrada y Salida:

input_dim = X_train_tensor.shape[1] # Datos de Entrada (22).
output_dim = len(np.unique(Y)) # Salidas Posibles (10).

class ModeloBatchNormalizationMomentum(nn.Module):

  def __init__(self):
      super(ModeloBatchNormalizationMomentum, self).__init__()

      self.capa_oculta1 = nn.Linear(input_dim, 128) # Capa Oculta (128 Neuronas).
      self.batch_normalization1 = nn.BatchNorm1d(128) # BatchNormalization.
      self.funcion_activacion1 = nn.ReLU() # RELU.

      self.capa_oculta2 = nn.Linear(128, 64) # Capa Oculta (64 Neuronas).
      self.batch_normalization2 = nn.BatchNorm1d(64) # BatchNormalization.
      self.funcion_activacion2 = nn.ReLU() # RELU.

      self.capa_oculta3 = nn.Linear(64, 32) # Capa Oculta (32 Neuronas).
      self.batch_normalization3 = nn.BatchNorm1d(32) # BatchNormalization.
      self.funcion_activacion3 = nn.ReLU() # RELU.

      self.capa_salida = nn.Linear(32, output_dim) # Capa de Salida.

  def forward (self, x): # Definición del Flujo de Datos del Modelo.

      # 1. Paso por la Primera Capa Oculta, Normalización y Aplicación de la RELU:

      x = self.capa_oculta1(x)
      x = self.batch_normalization1(x)
      x = self.funcion_activacion1(x)

      # 2. Paso por la Segunda Capa Oculta, Normalización y Aplicación de la RELU:

      x = self.capa_oculta2(x)
      x = self.batch_normalization2(x)
      x = self.funcion_activacion2(x)

      # 3. Paso por la Tercera Capa Oculta, Normalización y Aplicación de la RELU:

      x = self.capa_oculta3(x)
      x = self.batch_normalization3(x)
      x = self.funcion_activacion3(x)

      # 4. Paso por la Capa de Salida:

      x = self.capa_salida(x)
      return x

modelo_batch_normalization_momentum = ModeloBatchNormalizationMomentum() # Creación del Modelo.

In [33]:
# Definición de la Función de Coste y el Optimizador:

funcion_coste = nn.CrossEntropyLoss() # Compara las Predicciones del Modelo con los Valores Reales (Calcula el Error).
optimizador_SGD = optim.SGD(modelo_batch_normalization_momentum.parameters(), lr=0.01, momentum=0.9) # Actualiza los Parámetros (Descenso de Gradiente). Learning Rate Controla Cuánto se Ajustan los Parámetros (Controla el Aprendizaje). Momentum Acelera el Aprendizaje y Evita Oscilaciones.

In [34]:
# Entrenamiento del Modelo:

# Fijación de Semillas Aleatorias (Garantizar Resultados Comparables entre los Diferentes Modelos)

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Reinicio del Modelo y Optimizador (Evita que el Modelo siga Aprendiendo de Ejecuciones Anteriores):

modelo_batch_normalization_momentum = ModeloBatchNormalizationMomentum() # Modelo Reiniciado.
optimizador_SGD = optim.SGD(modelo_batch_normalization_momentum.parameters(), lr=0.01, momentum=0.9) # Optimizador Reiniciado.

# Fijación del Número de Veces que se Entrenará el Modelo:

numero_vueltas = 100

print ("Entrenamiento del Modelo:")

for epoch in range (numero_vueltas):
    modelo_batch_normalization_momentum.train()
    error_acumulado = 0.0 # Acumulador del Error de cada Vuelta (Acumula el Error de Todos los Batches en cada una de las Vueltas).

    for inputs, labels in Train_Loader: # Ejecución por cada Batch de Train.

        # 1. Resetear los Gradientes Calculados en Entrenamientos Anteriores:
        optimizador_SGD.zero_grad()

        # 2. Forward (Modelo Realiza Predicciones):
        outputs = modelo_batch_normalization_momentum(inputs)

        # 3. Cálculo del Error:
        error = funcion_coste(outputs, labels)

        # 4. Backward / Retropropagación (Cálculo de Gradientes para Ajustar los Parámetros):
        error.backward()

        # 5. Optimizar (Actualización de los Parámetros):
        optimizador_SGD.step()

        # 6. Acumulación del Error (Sumatorio del Error de cada Batch = Error Total de cada Vuelta):
        error_acumulado += error.item()

    print(f"Vuelta {epoch+1}/{numero_vueltas}, Error: {error_acumulado/len(Train_Loader):.4f}")

Entrenamiento del Modelo:
Vuelta 1/100, Error: 1.3656
Vuelta 2/100, Error: 0.9467
Vuelta 3/100, Error: 0.8816
Vuelta 4/100, Error: 0.8604
Vuelta 5/100, Error: 0.8204
Vuelta 6/100, Error: 0.8082
Vuelta 7/100, Error: 0.7804
Vuelta 8/100, Error: 0.7745
Vuelta 9/100, Error: 0.7815
Vuelta 10/100, Error: 0.7574
Vuelta 11/100, Error: 0.7638
Vuelta 12/100, Error: 0.7227
Vuelta 13/100, Error: 0.7251
Vuelta 14/100, Error: 0.7000
Vuelta 15/100, Error: 0.6938
Vuelta 16/100, Error: 0.6943
Vuelta 17/100, Error: 0.6950
Vuelta 18/100, Error: 0.6798
Vuelta 19/100, Error: 0.6595
Vuelta 20/100, Error: 0.6449
Vuelta 21/100, Error: 0.6905
Vuelta 22/100, Error: 0.6524
Vuelta 23/100, Error: 0.6329
Vuelta 24/100, Error: 0.6307
Vuelta 25/100, Error: 0.6356
Vuelta 26/100, Error: 0.6480
Vuelta 27/100, Error: 0.6023
Vuelta 28/100, Error: 0.5878
Vuelta 29/100, Error: 0.5934
Vuelta 30/100, Error: 0.6283
Vuelta 31/100, Error: 0.6033
Vuelta 32/100, Error: 0.5844
Vuelta 33/100, Error: 0.5955
Vuelta 34/100, Error: 0.58

In [35]:
# Evaluación del Modelo:

modelo_batch_normalization_momentum.eval()

# Creación de Contadores (Predicciones Correctas y Jugadores Totales Evaluados):

numero_aciertos = 0
numero_jugadores_evaluados = 0

with torch.no_grad():

     for inputs, labels in Test_Loader: # Ejecución por cada Batch de Test.

         # 1. Forward (Modelo Realiza Predicciones):
         outputs = modelo_batch_normalization_momentum(inputs)

         # 2. Selección de la Clase Predicha (Mayor Probabilidad):
         predicciones = torch.argmax(outputs, dim=1)

         # 3. Cálculo del Número de Predicciones Correctas (Predicciones = Etiquetas Reales):
         numero_aciertos += (predicciones == labels).sum().item()

         # 4. Cálculo del Número Total de Jugadores Evaluados:
         numero_jugadores_evaluados += labels.size(0)

# 5. Calculo del Accuracy (% Predicciones Correctas):
accuracy = 100 * numero_aciertos / numero_jugadores_evaluados

print(f"Precisión del Modelo: {accuracy:.2f}%")

Precisión del Modelo: 68.03%
